# Data Preprocessing & Feature Engineering

## Overview
This notebook handles the **second phase** of our vulnerability analysis pipeline: transforming raw CVE data into ML-ready features.

### Objectives
1. **Load Processed Data** from the EDA phase with proper validation
2. **Handle Missing Values** using domain-appropriate imputation strategies
3. **Feature Engineering** - create binary risk indicators and normalize scores
4. **Categorical Encoding** - convert categorical variables to numerical representations
5. **Feature Scaling** - normalize continuous variables for ML algorithms
6. **Data Validation** - ensure data quality and consistency before modeling

### Preprocessing Strategy
Our preprocessing approach focuses on:
- **Domain Knowledge**: Use cybersecurity expertise to guide imputation decisions
- **Data Consistency**: Maintain logical relationships between features
- **Model Readiness**: Prepare features optimized for machine learning algorithms
- **Interpretability**: Keep transformations transparent and reversible when possible

### Expected Outcomes
- Clean dataset with zero missing values
- Properly encoded categorical features
- Normalized numerical features in consistent scales
- Binary risk indicators for high-impact vulnerabilities
- Saved preprocessed data ready for baseline modeling

> **Rationale:** Systematic preprocessing ensures our models train on clean, consistent data while preserving the semantic meaning of cybersecurity features.


In [123]:
# Import required libraries for preprocessing and feature engineering
import pandas as pd                        # Data manipulation and analysis
import numpy as np                         # Numerical operations
import matplotlib.pyplot as plt            # Plotting and visualization
import seaborn as sns                      # Statistical visualization
import warnings                            # Warning control
from pathlib import Path                   # File path handling
from sklearn.preprocessing import (        # Scikit-learn preprocessing tools
    MinMaxScaler, OneHotEncoder, LabelEncoder
)
import os                                  # Operating system interface

# Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")


✅ All libraries imported successfully!


## 1. Data Loading and Validation

### Loading Processed Data
We load the cleaned dataset from the EDA phase, which includes:
- CVE records filtered to 2016+ for data quality
- Target variable based on Known Exploited Vulnerabilities
- Initial column pruning and deduplication completed


In [124]:
# Load the processed dataset from the EDA phase
input_path = '../../data/processed_vulnerabilities.csv'
print(f"📂 Loading processed dataset from: {input_path}")

# Verify file exists before loading
if not os.path.exists(input_path):
    print("❌ Error: Processed data file not found!")
    print("Please run the 0_Data_Loading_and_EDA.ipynb notebook first.")
    raise FileNotFoundError(f"Data file not found: {input_path}")

# Load the dataset
df = pd.read_csv(input_path)

# Validate the loaded data
print(f"✅ Data loaded successfully!")
print(f"📊 Dataset Overview:")
print(f"  Shape: {df.shape}")
print(f"  Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"  Columns: {list(df.columns)}")

# Verify critical columns exist
required_columns = ['id', 'target', 'year']
missing_columns = [col for col in required_columns if col not in df.columns]
if missing_columns:
    raise ValueError(f"Missing required columns: {missing_columns}")

# Display basic statistics
print(f"\n📈 Data Quality Check:")
print(f"  CVE records: {len(df):,}")
print(f"  Year range: {df['year'].min()}-{df['year'].max()}")
print(f"  Exploited CVEs: {df['target'].sum():,} ({df['target'].mean()*100:.2f}%)")
print(f"  Missing values: {df.isnull().sum().sum():,}")

print("\n🔍 First 3 rows:")
df.head(3)

📂 Loading processed dataset from: ../../data/processed_vulnerabilities.csv
✅ Data loaded successfully!
📊 Dataset Overview:
  Shape: (195522, 21)
  Memory usage: 118.56 MB
  Columns: ['id', 'baseScoreAv', 'exploitScoreAv', 'impactScoreAv', 'baseScoreMax', 'exploitScoreMax', 'impactScoreMax', 'version', 'baseSeverity', 'attackVector', 'attackComplexity', 'privilegesRequired', 'scope', 'confidentialityImpact', 'integrityImpact', 'availabilityImpact', 'numScores', 'agreement', 'userInteraction', 'target', 'year']

📈 Data Quality Check:
  CVE records: 195,522
  Year range: 2016-2025
  Exploited CVEs: 1,149 (0.59%)
  Missing values: 1,290

🔍 First 3 rows:


,id,baseScoreAv,exploitScoreAv,impactScoreAv,baseScoreMax,exploitScoreMax,impactScoreMax,version,baseSeverity,attackVector,attackComplexity,privilegesRequired,scope,confidentialityImpact,integrityImpact,availabilityImpact,numScores,agreement,userInteraction,target,year
0,CVE-2016-2978,3.3,1.8,1.4,3.3,1.8,1.4,3.0,LOW,LOCAL,LOW,LOW,UNCHANGED,LOW,NONE,NONE,1,0,NONE,0,2016
1,CVE-2016-2094,7.5,3.9,3.6,7.5,3.9,3.6,3.0,HIGH,NETWORK,LOW,NONE,UNCHANGED,NONE,NONE,HIGH,1,0,NONE,0,2016
2,CVE-2016-2997,5.4,2.3,2.7,5.4,2.3,2.7,3.0,MEDIUM,NETWORK,LOW,LOW,CHANGED,LOW,LOW,NONE,1,0,REQUIRED,0,2016


In [125]:
# Verify target variable distribution
print("🎯 Target Variable Analysis:")
target_counts = df['target'].value_counts().sort_index()
total = len(df)
print(f"  Not Exploited (0): {target_counts[0]:,} ({target_counts[0]/total*100:.2f}%)")
print(f"  Exploited (1): {target_counts[1]:,} ({target_counts[1]/total*100:.2f}%)")
print(f"  Imbalance ratio: {target_counts[0]/target_counts[1]:.1f}:1")

target_counts

🎯 Target Variable Analysis:
  Not Exploited (0): 194,373 (99.41%)
  Exploited (1): 1,149 (0.59%)
  Imbalance ratio: 169.2:1


target
0    194373
1      1149
Name: count, dtype: int64

## 2. Data Preparation and Feature Analysis

### Initial Data Preparation
Before preprocessing, we need to:
- Create a working copy of our data
- Analyze feature uniqueness and distribution
- Remove non-predictive columns (IDs, metadata)
- Understand missing value patterns for imputation strategy


In [126]:
# Create a working copy for preprocessing
# This preserves the original data for reference
data = df.copy()  # Use copy() to avoid unintended modifications
print(f"✅ Created working copy of data: {data.shape}")
print(f"📊 Working dataset ready for preprocessing")

✅ Created working copy of data: (195522, 21)
📊 Working dataset ready for preprocessing


In [127]:
# Analyze feature uniqueness to understand data diversity
print("🔍 Feature Uniqueness Analysis:")
print("This helps identify potential categorical vs numerical features")
print("=" * 60)

uniqueness = data.nunique().sort_values()
for col, unique_count in uniqueness.items():
    percentage = (unique_count / len(data)) * 100
    if unique_count <= 10:
        print(f"{col:<25}: {unique_count:>6} unique values ({percentage:>5.1f}%) - Categorical")
    elif unique_count <= 100:
        print(f"{col:<25}: {unique_count:>6} unique values ({percentage:>5.1f}%) - Mixed")
    else:
        print(f"{col:<25}: {unique_count:>6} unique values ({percentage:>5.1f}%) - Continuous")

print(f"\n📋 Detailed uniqueness breakdown:")
uniqueness

🔍 Feature Uniqueness Analysis:
This helps identify potential categorical vs numerical features
attackComplexity         :      2 unique values (  0.0%) - Categorical
userInteraction          :      2 unique values (  0.0%) - Categorical
scope                    :      2 unique values (  0.0%) - Categorical
target                   :      2 unique values (  0.0%) - Categorical
numScores                :      3 unique values (  0.0%) - Categorical
version                  :      3 unique values (  0.0%) - Categorical
privilegesRequired       :      3 unique values (  0.0%) - Categorical
attackVector             :      4 unique values (  0.0%) - Categorical
availabilityImpact       :      5 unique values (  0.0%) - Categorical
integrityImpact          :      5 unique values (  0.0%) - Categorical
confidentialityImpact    :      5 unique values (  0.0%) - Categorical
baseSeverity             :      5 unique values (  0.0%) - Categorical
year                     :     10 unique values (  0.

attackComplexity              2
userInteraction               2
scope                         2
target                        2
numScores                     3
version                       3
privilegesRequired            3
attackVector                  4
availabilityImpact            5
integrityImpact               5
confidentialityImpact         5
baseSeverity                  5
year                         10
agreement                    14
impactScoreMax               24
exploitScoreMax              36
baseScoreMax                 87
impactScoreAv               111
exploitScoreAv              115
baseScoreAv                 277
id                       195522
dtype: int64

## 3. Column Pruning and Missing Value Analysis

### Column Removal Strategy
Remove columns that are not useful for prediction:
- **Metadata**: IDs, descriptions, version info
- **Temporal**: Year (used for filtering, not prediction)
- **Redundant**: Vector strings, raw descriptions

### Missing Value Imputation Strategy
Our approach to handling missing values:
- **Categorical Features**: Impute with 'NONE' or most logical default
- **Risk-based Logic**: Use cybersecurity domain knowledge  
- **Consistency**: Maintain logical relationships between features


In [128]:
data['agreement'].value_counts()

agreement
0     163422
3       8351
2       7920
5       6240
4       5489
6       3050
7        790
8        207
9         27
10        14
12         7
11         3
16         1
14         1
Name: count, dtype: int64

In [129]:
data['numScores'].value_counts()

numScores
1    138137
2     57280
3       105
Name: count, dtype: int64

In [130]:
data.numScores.isnull().sum()

np.int64(0)

In [131]:
data['target'].value_counts()

target
0    194373
1      1149
Name: count, dtype: int64

## 4. Missing Value Imputation Strategy

### Domain-Driven Imputation Approach
For cybersecurity data, missing values often have meaningful interpretations:

#### **User Interaction Imputation**
- Missing → 'NONE': Assumes no user interaction required (most secure default)
- Rationale: Unknown interaction requirements default to most restrictive assumption

#### **Attack Vector Imputation** 
- Missing → 'NONE': Conservative approach for unknown attack vectors
- Rationale: Missing attack vector information treated as undefined/unknown

#### **Attack Complexity Imputation**
- Missing → 'LOW': Assumes attacks are easier (conservative security stance)  
- Rationale: Default to higher risk assumption when information is missing

#### **Privileges Required Imputation**
- Missing → 'NONE': Assumes no privileges needed (higher risk assumption)
- Rationale: Missing privilege info defaults to most accessible scenario

#### **Base Severity Imputation**
- Missing → 'NONE': Indicates unassigned/unknown severity
- Will handle in feature engineering phase


In [132]:
data.tail()

,id,baseScoreAv,exploitScoreAv,impactScoreAv,baseScoreMax,exploitScoreMax,impactScoreMax,version,baseSeverity,attackVector,attackComplexity,privilegesRequired,scope,confidentialityImpact,integrityImpact,availabilityImpact,numScores,agreement,userInteraction,target,year
195517,CVE-2025-28410,9.8,3.9,5.9,9.8,3.9,5.9,3.1,CRITICAL,NETWORK,LOW,NONE,UNCHANGED,HIGH,HIGH,HIGH,1,0,NONE,0,2025
195518,CVE-2025-28943,5.9,1.7,3.7,5.9,1.7,3.7,3.1,MEDIUM,NETWORK,LOW,HIGH,CHANGED,LOW,LOW,LOW,1,0,REQUIRED,0,2025
195519,CVE-2025-28406,9.8,3.9,5.9,9.8,3.9,5.9,3.1,CRITICAL,NETWORK,LOW,NONE,UNCHANGED,HIGH,HIGH,HIGH,1,0,NONE,0,2025
195520,CVE-2025-28902,4.3,2.8,1.4,4.3,2.8,1.4,3.1,MEDIUM,NETWORK,LOW,NONE,UNCHANGED,NONE,LOW,NONE,1,0,REQUIRED,0,2025
195521,CVE-2025-28144,6.5,3.9,2.5,6.5,3.9,2.5,3.1,MEDIUM,NETWORK,LOW,NONE,UNCHANGED,LOW,LOW,NONE,1,0,NONE,0,2025


In [133]:
data = data.drop(['year', 'version'], axis=1)

In [134]:
data.tail()

,id,baseScoreAv,exploitScoreAv,impactScoreAv,baseScoreMax,exploitScoreMax,impactScoreMax,baseSeverity,attackVector,attackComplexity,privilegesRequired,scope,confidentialityImpact,integrityImpact,availabilityImpact,numScores,agreement,userInteraction,target
195517,CVE-2025-28410,9.8,3.9,5.9,9.8,3.9,5.9,CRITICAL,NETWORK,LOW,NONE,UNCHANGED,HIGH,HIGH,HIGH,1,0,NONE,0
195518,CVE-2025-28943,5.9,1.7,3.7,5.9,1.7,3.7,MEDIUM,NETWORK,LOW,HIGH,CHANGED,LOW,LOW,LOW,1,0,REQUIRED,0
195519,CVE-2025-28406,9.8,3.9,5.9,9.8,3.9,5.9,CRITICAL,NETWORK,LOW,NONE,UNCHANGED,HIGH,HIGH,HIGH,1,0,NONE,0
195520,CVE-2025-28902,4.3,2.8,1.4,4.3,2.8,1.4,MEDIUM,NETWORK,LOW,NONE,UNCHANGED,NONE,LOW,NONE,1,0,REQUIRED,0
195521,CVE-2025-28144,6.5,3.9,2.5,6.5,3.9,2.5,MEDIUM,NETWORK,LOW,NONE,UNCHANGED,LOW,LOW,NONE,1,0,NONE,0


In [135]:
data.isnull().sum()

id                         0
baseScoreAv                0
exploitScoreAv             0
impactScoreAv              0
baseScoreMax               0
exploitScoreMax            0
impactScoreMax             0
baseSeverity             215
attackVector             215
attackComplexity         215
privilegesRequired       215
scope                    215
confidentialityImpact      0
integrityImpact            0
availabilityImpact         0
numScores                  0
agreement                  0
userInteraction          215
target                     0
dtype: int64

In [136]:
data.userInteraction.value_counts()

userInteraction
NONE        126855
REQUIRED     68452
Name: count, dtype: int64

In [137]:
data.userInteraction = data.userInteraction.fillna('NONE')

In [138]:
data.attackVector.value_counts()

attackVector
NETWORK             140872
LOCAL                46741
ADJACENT_NETWORK      5661
PHYSICAL              2033
Name: count, dtype: int64

In [139]:
data.attackVector = data.attackVector.fillna('NONE')

In [140]:
data.attackVector.isnull().sum()

np.int64(0)

In [141]:
data.attackComplexity.value_counts()

attackComplexity
LOW     177985
HIGH     17322
Name: count, dtype: int64

In [142]:
data.attackComplexity = data.attackComplexity.fillna('LOW')

In [143]:
data.attackComplexity.isnull().sum()

np.int64(0)

In [144]:
data.privilegesRequired.value_counts()

privilegesRequired
NONE    112148
LOW      64365
HIGH     18794
Name: count, dtype: int64

In [145]:
data.privilegesRequired = data.privilegesRequired.fillna('NONE')

In [146]:
data.baseSeverity.value_counts()

baseSeverity
MEDIUM      87398
HIGH        76641
CRITICAL    24645
LOW          6606
NONE           17
Name: count, dtype: int64

In [147]:
data.baseSeverity.fillna('NONE', inplace=True)

## 5. Feature Engineering - Risk Score Binarization

### High-Risk Threshold Strategy  
We convert CVSS scores to binary indicators using a threshold of 7.0 (High/Critical severity):

#### **Rationale for 7.0 Threshold**
- **Industry Standard**: CVSS 7.0+ is widely considered "High" severity
- **Risk Management**: Helps prioritize vulnerabilities requiring immediate attention  
- **Model Interpretability**: Binary features are easier to interpret than continuous scores
- **Class Balance**: May help with the extreme class imbalance in our target

#### **Features to Binarize**
- `baseScoreAv`, `exploitScoreAv`, `impactScoreAv`: Average scores across versions
- `baseScoreMax`, `exploitScoreMax`, `impactScoreMax`: Maximum scores across versions


In [148]:
data.baseSeverity.value_counts()

baseSeverity
MEDIUM      87398
HIGH        76641
CRITICAL    24645
LOW          6606
NONE          232
Name: count, dtype: int64

In [149]:
data.scope.value_counts()

scope
UNCHANGED    154647
CHANGED       40660
Name: count, dtype: int64

In [150]:
data.scope = data.scope.fillna('UNCHANGED')

In [151]:
data.isnull().sum()

id                       0
baseScoreAv              0
exploitScoreAv           0
impactScoreAv            0
baseScoreMax             0
exploitScoreMax          0
impactScoreMax           0
baseSeverity             0
attackVector             0
attackComplexity         0
privilegesRequired       0
scope                    0
confidentialityImpact    0
integrityImpact          0
availabilityImpact       0
numScores                0
agreement                0
userInteraction          0
target                   0
dtype: int64

## 6. Feature Scaling for Continuous Variables

### MinMax Scaling Strategy
We apply MinMax scaling to continuous variables to normalize them to [0,1] range:

#### **Features to Scale**
- `numScores`: Number of CVSS score versions available
- `agreement`: Score agreement/consensus metric

#### **Benefits of MinMax Scaling**
- **Algorithm Performance**: Many ML algorithms perform better with normalized features
- **Gradient Convergence**: Faster convergence for gradient-based optimizers
- **Feature Importance**: Prevents scale bias in feature importance calculations
- **Interpretability**: [0,1] range is intuitive and preserves relative relationships


In [152]:
data.tail()

,id,baseScoreAv,exploitScoreAv,impactScoreAv,baseScoreMax,exploitScoreMax,impactScoreMax,baseSeverity,attackVector,attackComplexity,privilegesRequired,scope,confidentialityImpact,integrityImpact,availabilityImpact,numScores,agreement,userInteraction,target
195517,CVE-2025-28410,9.8,3.9,5.9,9.8,3.9,5.9,CRITICAL,NETWORK,LOW,NONE,UNCHANGED,HIGH,HIGH,HIGH,1,0,NONE,0
195518,CVE-2025-28943,5.9,1.7,3.7,5.9,1.7,3.7,MEDIUM,NETWORK,LOW,HIGH,CHANGED,LOW,LOW,LOW,1,0,REQUIRED,0
195519,CVE-2025-28406,9.8,3.9,5.9,9.8,3.9,5.9,CRITICAL,NETWORK,LOW,NONE,UNCHANGED,HIGH,HIGH,HIGH,1,0,NONE,0
195520,CVE-2025-28902,4.3,2.8,1.4,4.3,2.8,1.4,MEDIUM,NETWORK,LOW,NONE,UNCHANGED,NONE,LOW,NONE,1,0,REQUIRED,0
195521,CVE-2025-28144,6.5,3.9,2.5,6.5,3.9,2.5,MEDIUM,NETWORK,LOW,NONE,UNCHANGED,LOW,LOW,NONE,1,0,NONE,0


In [153]:
data.exploitScoreAv.max()

np.float64(10.0)

In [154]:
# bianary encode each of the rows other than id as 1 if 7+ and 0 otherwise

columns_to_encode = ['baseScoreAv', 'exploitScoreAv', 'impactScoreAv', 'baseScoreMax', 'exploitScoreMax', 'impactScoreMax']

for col in columns_to_encode:
  data[col] = data[col].apply(lambda x: 1 if x >= 7 else 0)


## 7. Categorical Feature Encoding

### One-Hot Encoding Strategy
Converting categorical CVSS features to binary indicators for ML compatibility:

#### **Categorical Features to Encode**
- `baseSeverity`: LOW, MEDIUM, HIGH, CRITICAL, NONE
- `attackVector`: NETWORK, ADJACENT_NETWORK, LOCAL, PHYSICAL  
- `attackComplexity`: LOW, HIGH
- `privilegesRequired`: NONE, LOW, HIGH
- `scope`: UNCHANGED, CHANGED
- `confidentialityImpact`: HIGH, LOW, NONE
- `integrityImpact`: HIGH, LOW, NONE  
- `availabilityImpact`: HIGH, LOW, NONE
- `userInteraction`: NONE, REQUIRED

#### **One-Hot Encoding Benefits**
- **No Ordinal Assumptions**: Prevents model from assuming incorrect ordering
- **Model Compatibility**: Works with all ML algorithms
- **Feature Interpretability**: Each category becomes an interpretable binary feature
- **Avoiding Dummy Variable Trap**: Use `drop='first'` to remove redundancy


In [155]:
data.head()

,id,baseScoreAv,exploitScoreAv,impactScoreAv,baseScoreMax,exploitScoreMax,impactScoreMax,baseSeverity,attackVector,attackComplexity,privilegesRequired,scope,confidentialityImpact,integrityImpact,availabilityImpact,numScores,agreement,userInteraction,target
0,CVE-2016-2978,0,0,0,0,0,0,LOW,LOCAL,LOW,LOW,UNCHANGED,LOW,NONE,NONE,1,0,NONE,0
1,CVE-2016-2094,1,0,0,1,0,0,HIGH,NETWORK,LOW,NONE,UNCHANGED,NONE,NONE,HIGH,1,0,NONE,0
2,CVE-2016-2997,0,0,0,0,0,0,MEDIUM,NETWORK,LOW,LOW,CHANGED,LOW,LOW,NONE,1,0,REQUIRED,0
3,CVE-2016-2217,0,0,0,0,0,0,MEDIUM,NETWORK,LOW,NONE,UNCHANGED,LOW,NONE,NONE,1,0,NONE,0
4,CVE-2016-2352,1,0,0,1,0,0,HIGH,NETWORK,LOW,LOW,UNCHANGED,HIGH,HIGH,HIGH,1,0,NONE,0


In [156]:
from sklearn.preprocessing import MinMaxScaler

cols_to_scale = ['numScores', 'agreement']

scaler = MinMaxScaler()
data[cols_to_scale] = scaler.fit_transform(data[cols_to_scale])

In [157]:
data.head()

,id,baseScoreAv,exploitScoreAv,impactScoreAv,baseScoreMax,exploitScoreMax,impactScoreMax,baseSeverity,attackVector,attackComplexity,privilegesRequired,scope,confidentialityImpact,integrityImpact,availabilityImpact,numScores,agreement,userInteraction,target
0,CVE-2016-2978,0,0,0,0,0,0,LOW,LOCAL,LOW,LOW,UNCHANGED,LOW,NONE,NONE,0.0,0.0,NONE,0
1,CVE-2016-2094,1,0,0,1,0,0,HIGH,NETWORK,LOW,NONE,UNCHANGED,NONE,NONE,HIGH,0.0,0.0,NONE,0
2,CVE-2016-2997,0,0,0,0,0,0,MEDIUM,NETWORK,LOW,LOW,CHANGED,LOW,LOW,NONE,0.0,0.0,REQUIRED,0
3,CVE-2016-2217,0,0,0,0,0,0,MEDIUM,NETWORK,LOW,NONE,UNCHANGED,LOW,NONE,NONE,0.0,0.0,NONE,0
4,CVE-2016-2352,1,0,0,1,0,0,HIGH,NETWORK,LOW,LOW,UNCHANGED,HIGH,HIGH,HIGH,0.0,0.0,NONE,0


In [158]:
from sklearn.preprocessing import OneHotEncoder

# Define the list of categorical columns to encode
categorical_cols = ['baseSeverity',	'attackVector',	'attackComplexity',
                    'privilegesRequired',	'scope', 'confidentialityImpact',
                    'integrityImpact',	'availabilityImpact', 'userInteraction']

# Convert all categorical columns to string
data[categorical_cols] = data[categorical_cols].astype(str)

# Create encoder instance
encoder = OneHotEncoder(drop='first', sparse_output=False)  # drop='first' if you want to avoid dummy variable trap

# Fit and transform the data
encoded_array = encoder.fit_transform(data[categorical_cols])

# Get column names
encoded_cols = encoder.get_feature_names_out(categorical_cols)

# Create encoded DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoded_cols)

# Combine with original DataFrame (excluding original categorical columns)
data_final = pd.concat([data.drop(columns=categorical_cols).reset_index(drop=True),
                      encoded_df.reset_index(drop=True)], axis=1)

print(data_final)

                    id  baseScoreAv  exploitScoreAv  impactScoreAv  \
0        CVE-2016-2978            0               0              0   
1        CVE-2016-2094            1               0              0   
2        CVE-2016-2997            0               0              0   
3        CVE-2016-2217            0               0              0   
4        CVE-2016-2352            1               0              0   
...                ...          ...             ...            ...   
195517  CVE-2025-28410            1               0              0   
195518  CVE-2025-28943            0               0              0   
195519  CVE-2025-28406            1               0              0   
195520  CVE-2025-28902            0               0              0   
195521  CVE-2025-28144            0               0              0   

        baseScoreMax  exploitScoreMax  impactScoreMax  numScores  agreement  \
0                  0                0               0        0.0        0.0   
1

In [159]:
pd.set_option('display.max_columns', None)
data_final.head()

,id,baseScoreAv,exploitScoreAv,impactScoreAv,baseScoreMax,exploitScoreMax,impactScoreMax,numScores,agreement,target,baseSeverity_HIGH,baseSeverity_LOW,baseSeverity_MEDIUM,baseSeverity_NONE,attackVector_LOCAL,attackVector_NETWORK,attackVector_NONE,attackVector_PHYSICAL,attackComplexity_LOW,privilegesRequired_LOW,privilegesRequired_NONE,scope_UNCHANGED,confidentialityImpact_HIGH,confidentialityImpact_LOW,confidentialityImpact_NONE,confidentialityImpact_PARTIAL,integrityImpact_HIGH,integrityImpact_LOW,integrityImpact_NONE,integrityImpact_PARTIAL,availabilityImpact_HIGH,availabilityImpact_LOW,availabilityImpact_NONE,availabilityImpact_PARTIAL,userInteraction_REQUIRED
0,CVE-2016-2978,0,0,0,0,0,0,0.0,0.0,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,CVE-2016-2094,1,0,0,1,0,0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,CVE-2016-2997,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,CVE-2016-2217,0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,CVE-2016-2352,1,0,0,1,0,0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### Insights

* All remaining features are numeric, either continuous (scaled) or sparse binary indicators generated from categorical factors.
* Re‑inspection after preprocessing shows **zero missing values** – the dataset is now ML‑ready.
* Feature distributions look approximately centred in `[0,1]`, which speeds up gradient‑based learners.


## 8. Data Export for Modeling Phase

Saving the fully preprocessed dataset for the baseline modeling phase. This ensures our modeling notebooks start with clean, ML-ready data.


In [160]:
# Save the preprocessed dataset for the modeling phase
output_path = '../../data/processed_vulnerabilities_features.csv'
print(f"💾 Saving preprocessed dataset to: {output_path}")

# Create output directory if it doesn't exist
import os
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Save the preprocessed dataset
data_final.to_csv(output_path, index=False)

# Verify the save operation and provide comprehensive summary
saved_size = os.path.getsize(output_path) / 1024**2  # Size in MB
print(f"✅ Preprocessed dataset saved successfully!")

print(f"\n📊 Final Preprocessing Summary:")
print(f"  Records: {len(data_final):,}")
print(f"  Features: {len(data_final.columns):,}")
print(f"  File size: {saved_size:.2f} MB")
print(f"  Missing values: {data_final.isnull().sum().sum()}")
print(f"  Exploited CVEs: {data_final['target'].sum():,} ({(data_final['target'].mean()*100):.2f}%)")

# Feature type breakdown
feature_cols = [col for col in data_final.columns if col not in ['id', 'target']]
print(f"  Feature columns: {len(feature_cols)}")

# Identify binary vs continuous features
binary_features = []
continuous_features = []
for col in feature_cols:
    unique_vals = data_final[col].nunique()
    if unique_vals == 2 and set(data_final[col].unique()).issubset({0, 1, 0.0, 1.0}):
        binary_features.append(col)
    else:
        continuous_features.append(col)

print(f"  Binary features: {len(binary_features)}")
print(f"  Continuous features: {len(continuous_features)}")

print(f"\n🎯 Next Steps:")
print(f"  1. Open notebook: 2_Baseline_Modeling.ipynb")
print(f"  2. Load data from: {output_path}")
print(f"  3. Split data and train baseline models")
print(f"  4. Evaluate performance with proper metrics")

print(f"\n✨ Preprocessing Phase Complete! ✨")


💾 Saving preprocessed dataset to: ../../data/processed_vulnerabilities_features.csv
✅ Preprocessed dataset saved successfully!

📊 Final Preprocessing Summary:
  Records: 195,522
  Features: 35
  File size: 25.56 MB
  Missing values: 0
  Exploited CVEs: 1,149 (0.59%)
  Feature columns: 33
  Binary features: 31
  Continuous features: 2

🎯 Next Steps:
  1. Open notebook: 2_Baseline_Modeling.ipynb
  2. Load data from: ../../data/processed_vulnerabilities_features.csv
  3. Split data and train baseline models
  4. Evaluate performance with proper metrics

✨ Preprocessing Phase Complete! ✨
